In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
pip install requests beautifulsoup4 nltk pandas openpyxl

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the title and article text (adjust selectors as needed)
    title = soup.find('h1').get_text(strip=True)
    paragraphs = soup.find_all('p')
    article_text = '\n'.join([para.get_text(strip=True) for para in paragraphs])

    return title + "\n" + article_text

def save_article_text(url_id, text):
    with open(f'extracted_articles/{url_id}.txt', 'w', encoding='utf-8') as file:
        file.write(text)

def main():
    # Read the input URLs
    input_df = pd.read_excel('/content/drive/MyDrive/Test Assignment/20211030 Test Assignment/Input.xlsx')

    # Ensure the output directory exists
    os.makedirs('extracted_articles', exist_ok=True)

    # Extract and save article text for each URL
    for _, row in input_df.iterrows():
        url_id = row['URL_ID']
        url = row['URL']
        try:
            article_text = extract_article_text(url)
            save_article_text(url_id, article_text)
        except Exception as e:
            print(f"Failed to extract {url_id} - {url}: {e}")

if __name__ == '__main__':
    main()

Failed to extract blackassign0036 - https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/: 'NoneType' object has no attribute 'get_text'
Failed to extract blackassign0049 - https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/: 'NoneType' object has no attribute 'get_text'


In [5]:
pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.2 MB/s eta 0:00:00


In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import textstat
import pandas as pd
import os
import re

nltk.download('punkt')
nltk.download('stopwords')

# Load stopwords
stopwords = set(stopwords.words('english'))

# Function to read files with encoding handling
def read_file_with_encoding_handling(file_path):
    for encoding in ['utf-8', 'latin-1']:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read().split()
        except UnicodeDecodeError:
            continue
    raise ValueError(f"Failed to read file with known encodings: {file_path}")

# Load positive and negative words
positive_words = set(read_file_with_encoding_handling('/content/drive/MyDrive/Test Assignment/20211030 Test Assignment/MasterDictionary/positive-words.txt'))
negative_words = set(read_file_with_encoding_handling('/content/drive/MyDrive/Test Assignment/20211030 Test Assignment/MasterDictionary/negative-words.txt'))

def compute_variables(text):
    words = word_tokenize(text.lower())
    sentences = sent_tokenize(text)

    # Positive and negative score
    positive_score = sum(1 for word in words if word in positive_words)
    negative_score = sum(1 for word in words if word in negative_words)

    # Polarity and subjectivity
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(words) + 0.000001)

    # Sentence lengths and complexity
    avg_sentence_length = sum(len(word_tokenize(sentence)) for sentence in sentences) / len(sentences)
    complex_words = [word for word in words if textstat.syllable_count(word) > 2]
    percentage_of_complex_words = len(complex_words) / len(words)
    fog_index = textstat.gunning_fog(text)
    avg_words_per_sentence = len(words) / len(sentences)

    # Other metrics
    complex_word_count = len(complex_words)
    word_count = len(words)
    syllable_per_word = sum(textstat.syllable_count(word) for word in words) / len(words)
    personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.I))
    avg_word_length = sum(len(word) for word in words) / len(words)

    return {
        'positive_score': positive_score,
        'negative_score': negative_score,
        'polarity_score': polarity_score,
        'subjectivity_score': subjectivity_score,
        'avg_sentence_length': avg_sentence_length,
        'percentage_of_complex_words': percentage_of_complex_words,
        'fog_index': fog_index,
        'avg_words_per_sentence': avg_words_per_sentence,
        'complex_word_count': complex_word_count,
        'word_count': word_count,
        'syllable_per_word': syllable_per_word,
        'personal_pronouns': personal_pronouns,
        'avg_word_length': avg_word_length
    }

def main():
    input_df = pd.read_excel('/content/drive/MyDrive/Test Assignment/20211030 Test Assignment/Input.xlsx')
    output_df = pd.read_excel('/content/drive/MyDrive/Test Assignment/20211030 Test Assignment/Output Data Structure.xlsx')

    results = []

    for _, row in input_df.iterrows():
        url_id = row['URL_ID']
        file_path = f'extracted_articles/{url_id}.txt'
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                variables = compute_variables(text)
                results.append([url_id, row['URL'], *variables.values()])
        else:
            print(f"File not found: {file_path}")

    # Save results to DataFrame
    result_df = pd.DataFrame(results, columns=output_df.columns)
    result_df.to_excel('/content/drive/MyDrive/Test Assignment/20211030 Test Assignment/Output Data Structure.xlsx', index=False)

if __name__ == '__main__':
    main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


File not found: extracted_articles/blackassign0036.txt
File not found: extracted_articles/blackassign0049.txt
